In [ ]:
import dpln
from dpln import nn, optim
from dpln.nn import functional as F
from dpln.util import load_pickle
from dpln.data import Dataset, DataLoader

import numpy as np
from matplotlib import pyplot as plt

print(dpln.__version__)

In [ ]:
mnist = load_pickle("mnist.pkl")

In [ ]:
class MnistDataset(Dataset):
    def __init__(self, images, labels, transform=None, transform_target=None):
        if len(images) != len(labels):
            raise ValueError("length of images and labels must equal")
        self.images = images
        self.labels = labels
        self.transform = transform
        self.transform_target = transform_target

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img = self.images[idx]
        lab = self.labels[idx]
        if self.transform:
            img = self.transform(img)
        if self.transform_target:
            lab = self.transform_target(lab)
        return img, lab


In [ ]:
def transform(img):
    return img.reshape(28 * 28) / 256


def transform_target(lab):
    return np.array([lab])


In [ ]:
train_ds = MnistDataset(mnist["train_images"], mnist["train_labels"],
                        transform=transform, transform_target=transform_target)
test_ds = MnistDataset(mnist["test_images"], mnist["test_labels"],
                       transform=transform, transform_target=transform_target)


In [ ]:
def batch_fn(batch):
    images = []
    labels = []
    for i in batch:
        images.append(i[0][np.newaxis, :])
        labels.append(i[1])
    b_img = dpln.Tensor(np.concatenate(images))
    b_lab = dpln.Tensor(np.concatenate(labels).squeeze())
    return b_img, b_lab


In [ ]:
train_ld = DataLoader(train_ds, 64, batch_fn)
test_ld = DataLoader(test_ds, 1000, batch_fn)

In [ ]:
sample_image_batch, sample_label_batch = next(iter(train_ld))
sample_image = sample_image_batch.numpy()[0].reshape(1, 784)
sample_label = sample_label_batch.numpy()[0].reshape(1, 1)

In [ ]:
plt.imshow(sample_image.reshape(28, 28))

In [ ]:
print(sample_label)

In [ ]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.l0 = nn.Linear(28 * 28, 64)
        self.l1 = nn.Linear(64, 64)
        self.l2 = nn.Linear(64, 32)
        self.final = nn.Linear(32, 10)

    def forward(self, x: dpln.Tensor) -> dpln.Tensor:
        x = self.l0(x)
        x = F.sigmoid(x)
        x = self.l1(x)
        x = F.sigmoid(x)
        x = self.l2(x)
        x = F.sigmoid(x)
        x = self.final(x)
        x = F.log_softmax(x, dim=1)
        return x


model = Model()


In [ ]:
print(model)

In [ ]:
optm = optim.SGD(model.parameters(), 1)

In [ ]:
train_losses = []

In [ ]:
def train(model: nn.Module, epoch: int, train_loader, optm):
    train_loss = 0
    for batch_idx, (inp, tar) in enumerate(train_loader):
        optm.zero_grad()
        pre = model(inp)
        loss = F.nll_loss(pre, tar)
        loss.backward()
        optm.step()
        loss_v = loss.item()
        train_loss += loss_v
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(inp), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
    train_loss /= len(train_loader.dataset)
    train_losses.append(train_loss)


In [ ]:
test_losses = []
test_accu = []

In [ ]:
def test(model, test_loader):
    test_loss = 0
    correct = 0
    for inp, tar in test_loader:
        out = model(inp)
        test_loss += F.nll_loss(out, tar, reduction='sum').item()
        pred = out.argmax(dim=1, keepdim=True)
        correct += pred.eq(tar.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    test_losses.append(test_loss)
    test_a = 100. * correct / len(test_loader.dataset)
    test_accu.append(test_a)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        test_a))


In [ ]:
for i in range(20):
    train(i, model, train_ld, optm)
    test(model, test_ld)


In [ ]:
plt.figure()
plt.title("loss")
plt.plot(train_losses, label='train_loss')
plt.plot(test_losses, label='test_loss')
plt.legend()
plt.show()

In [ ]:
plt.figure()
plt.title("accuracy")
plt.plot(test_accu, label='test_accu')
plt.show()